In [677]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from datetime import datetime

In [ ]:
twitterdf = pd.concat(
    [
        pd.read_json("/Users/zoey/Downloads/corona-out-2", lines=True),
        pd.read_json("/Users/zoey/Downloads/corona-out-3", lines=True)
    ]
)

# Drop duplicates on user_id
twitterdf["user_id"] = twitterdf["user"].apply(lambda x: x["id"])
twitterdf.drop_duplicates(subset = ["user_id"], keep = "last", inplace = True)

# Reset indices
twitterdf.reset_index(inplace = True, drop = True)


In [ ]:
twitterdf.head()

In [ ]:
twitterdf['user'][1]

In [ ]:
usersdf.isna().sum()

In [ ]:
usersdf.fillna(value = None, inplace = True)

In [ ]:
import psycopg2
from psycopg2 import extensions
from configparser import ConfigParser

import os
print(os.getcwd())

config = ConfigParser()
config.read('../../config.ini')

conn = psycopg2.connect(
    **dict(config.items('postgres'))
)

cur = conn.cursor()

In [ ]:
table1Columns = [
    'user_id',
    'name',
    'screen_name',
    'url',
    'location',
    'followers_count',
    'friends_count',
    'listed_count',
    'favourites_count',
    'statuses_count',
    'created_at',
    'description',
    'language',
    'verified',
    'profile_image_url',
    'profile_background_image_url',
    'default_profile',
    'default_profile_image'
]

table1Columns = ', '.join(table1Columns)
table1Columns

# [val for val in usersdf.iloc[1,1:].values.tolist() if val != None
table1 = usersdf[[
    'id_str',
    'name',
    'screen_name',
    'url',
    'location',
    'followers_count',
    'friends_count',
    'listed_count',
    'favourites_count',
    'statuses_count',
    'created_at',
    'description',
    'lang',
    'verified',
    'profile_image_url_https',
    'profile_background_image_url_https',
    'default_profile',
    'default_profile_image',
]]

table1Values = str(table1.iloc[1,:].values.tolist()).strip('][').replace("None", "NULL")

cur.execute(
    f"""
        INSERT INTO user_profile ({table1Columns})
        VALUES ({table1Values})
    """
)

conn.commit()
conn.close()


In [ ]:


# check if any cell in the DataFrame contains the phrase
is_present = table1.dropna().applymap(lambda x: 'Official page of the constantly evolving sound' in x).any().any()

In [ ]:
table1Values = str(table1.iloc[1,:].values.tolist()).strip('][')
f""" INSERT INTO user_profile ({table1Columns}) VALUES ({table1Values})"""

In [ ]:
cur.execute(
    """
    CREATE TABLE user_profile 
    (
        user_id TEXT PRIMARY KEY, 
        name TEXT, 
        screen_name VARCHAR(15), 
        url TEXT NULL , 
        location TEXT NULL, 
        followers_count INTEGER NULL,
        friends_count INTEGER, 
        listed_count INTEGER, 
        favourites_count INTEGER, 
        statuses_count INTEGER, 
        created_at VARCHAR(30), 
        description TEXT, 
        language CHAR(4), 
        verified BOOLEAN, 
        profile_image_url TEXT,
        profile_background_image_url TEXT NULL,
        default_profile BOOLEAN NULL,
        default_profile_image BOOLEAN NULL
    )
    """
)



for col in table1.select_dtypes(exclude=['int', 'int64', 'bool']).columns.tolist():
    table1[col] = table1[col].apply(lambda x: x.encode('ascii', 'ignore').decode('utf-8') if bool(x) else x)
    table1[col] = table1[col].apply(lambda x: extensions.adapt(x).getquoted().decode('utf-8'))
    table1[col] = table1[col].apply(lambda x: x.replace("'","").replace("%","%%") if bool(x) else x)

a = table1.values.tolist()
for _value in a:
    _row = str(
        _value
    ).strip(
        ']['
    ).replace(
        "None", 
        "NULL"
    )
        
    cur.execute(
        f"""
            INSERT INTO 
                user_profile ({table1Columns})
            VALUES 
                ({_row})
        """
    )

In [ ]:
conn.commit()
conn.close()

In [ ]:
print(table1Columns)
print('\n')
print(_value)

In [ ]:
f"""
            INSERT INTO 
                user_profile ({table1Columns})
            VALUES 
                ({_row})
        """

In [ ]:
table1.iloc[1,:].values.tolist()

In [ ]:
twitterdf.isna().sum()

In [ ]:
reply_columns=['reply_tweet_id','tweet_id']
quoted_tweet_columns=['quoted_tweet_id','tweet_id']
retweets_columns=['retweet_id','tweet_id']
tweets_columns=['tweet_id','original_tweet_flag','retweet_flag','quoted_tweet_flag','reply_tweet_flag']

reply_columns=', '.join(reply_columns)
quoted_tweet_columns=', '.join(quoted_tweet_columns)
retweets_columns=', '.join(retweets_columns)
tweets_columns=', '.join(tweets_columns)

In [ ]:
twitterdf['retweet_flag'] = np.where(
    twitterdf['retweeted_status'].isna(), 
    0, 
    1
)

twitterdf['quoted_tweet_flag'] = np.where(
    twitterdf['quoted_status_id_str'].isna(), 
    0, 
    1
)

twitterdf['reply_tweet_flag'] = np.where(
    twitterdf['in_reply_to_user_id_str'].isna(), 
    0, 
    1
)

twitterdf['original_tweet_flag'] = np.where(
    (twitterdf['retweet_flag'] == 0) & 
    (twitterdf['quoted_tweet_flag'] == 0) & 
    (twitterdf['reply_tweet_flag'] == 0),
    1,
    0
)

In [ ]:
twitterdf[twitterdf.quoted_tweet_flag == 1][twitterdf.retweet_flag == 1]

In [ ]:
retweetdf = twitterdf[
    twitterdf.retweeted_status.notna()
]

retweetdf['tweet_id'] = retweetdf['retweeted_status'].apply(
    lambda x: x["id_str"]
)

retweetdf[[['id_str','tweet_id']]

In [ ]:
table2Columns=twitterdf.loc[
    twitterdf.in_reply_to_status_id_str.notna(),
    ['in_reply_to_status_id_str','id_str']
]

quoted_tweetsdf=twitterdf.loc[
    twitterdf.quoted_status_id_str.notna(),
    ['quoted_status_id_str','id_str']
]


# tweetsdf=twitterdf.loc[
#     twitterdf.quoted_status_id_str.notna(),
#     ['id_str','quoted_status_id_str']
# ]
tweets_columns=['tweet_id','original_tweet_flag','retweet_flag','quoted_tweet_flag','reply_tweet_flag']


In [ ]:
reply_val = str(replydf.iloc[0,:].values.tolist()).strip('][')
quoted_tweets_val = str(quoted_tweetsdf.iloc[0,:].values.tolist()).strip('][')


In [ ]:
f"""
        INSERT INTO user_profile ({retweets_columns})
        VALUES ({retweets_val})

    """

# Non Relational DB

In [716]:
tdf=twitterdf[[
'id_str',
'created_at',
'text',
'source',
'user_id',
'truncated',
'lang',
'quote_count',
'reply_count',
'retweet_count',
'favorite_count',
'favorited',
'retweeted',
'possibly_sensitive',
'withheld_in_countries',
'place',
'entities',
'extended_entities',
'quoted_status',
'retweeted_status'
]
]

In [717]:
tdf['place'] = tdf['place'].map(str)
# tdf['entities'] = tdf['entities'].fillna(value = dict())
tdf['extended_entities'] = tdf['extended_entities'].map(str)
tdf['quoted_status'] = tdf['quoted_status'].map(str)
tdf['retweeted_status'] = tdf['retweeted_status'].map(str)

/var/folders/w3/d1c_m0dx0mv20q7033lt172h0000gn/T/ipykernel_9298/1685441906.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdf['place'] = tdf['place'].map(str)
/var/folders/w3/d1c_m0dx0mv20q7033lt172h0000gn/T/ipykernel_9298/1685441906.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdf['extended_entities'] = tdf['extended_entities'].map(str)
/var/folders/w3/d1c_m0dx0mv20q7033lt172h0000gn/T/ipykernel_9298/1685441906.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [715]:
# tdf[tdf.entities == dict()]

,id_str,created_at,text,source,user_id,truncated,lang,quote_count,reply_count,retweet_count,favorite_count,favorited,retweeted,possibly_sensitive,withheld_in_countries,place,entities,extended_entities,quoted_status,retweeted_status


In [687]:
tdf['created_at'] = tdf['created_at'].map()

TypeError: Series.map() missing 1 required positional argument: 'arg'

In [ ]:
tdf.to_json('~/a.json')

In [718]:
#!pip install elasticsearch
from elasticsearch import Elasticsearch
from configparser import ConfigParser

import os
print(os.getcwd())

config = ConfigParser()
config.read('../../config.ini')


/Users/zoey/694Team14Dbms2023/notebooks/vg422


['../../config.ini']

In [719]:
# es = Elasticsearch()
# es = Elasticsearch(
#     hosts=[{'host': '127.0.0.1', 'port': 9200}]
#     http_auth=('username', 'password')
# )

esConfig = dict(config.items('elasticsearch'))

# es = Elasticsearch(
#     hosts=[{'host': esConfig['host'], 'port': esConfig['port']}],
#     http_auth=(esConfig['elastic_username'], esConfig['elastic_password'])
# )

# # cur = conn.cursor()
client = Elasticsearch(
    f"http://{esConfig['host']}:{esConfig['port']}",
    basic_auth = (esConfig['elastic_username'], esConfig['elastic_password']),
    max_retries=3,
    retry_on_timeout=True
)

# # Successful response!
client.info()

/var/folders/w3/d1c_m0dx0mv20q7033lt172h0000gn/T/ipykernel_9298/665178185.py:23: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.14/security-minimal-setup.html to enable security.
  client.info()


ObjectApiResponse({'name': '890f001cd056', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'tJancf-PTCCF5Hd-3T-57g', 'version': {'number': '7.14.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'dd5a0a2acaa2045ff9624f3729fc8a6f40835aa1', 'build_date': '2021-07-29T20:49:32.864135063Z', 'build_snapshot': False, 'lucene_version': '8.9.0', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'})

In [ ]:
health = client.cluster.health()

# get number of nodes and shards
num_nodes = health['number_of_nodes']
num_shards = health['active_shards']

In [ ]:
index_names = ['twitterdata1','twitterdata2','twitterdata3']
for index_name in index_names:
    client.indices.create(index=index_name)


In [720]:
json_data = tdf.to_json(orient='records')

In [ ]:
# import json
# docs=json.dumps(docs)

In [721]:
import json
docs=json.loads(json_data)
print(type(docs))
# a[0]

<class 'list'>


In [ ]:
# from configparser import ConfigParser

# config = ConfigParser()
# config.read('../../config.ini')

# conn = Elasticsearch(
#     **dict(config.items('postgres'))
# )

In [722]:
doc_count = 0
base_index_name='twitter_data'
max_docs_per_index=1000

# Loop through the documents and index them
# for doc in docs[0:50]:
for doc in docs:
    # Compute the index name based on the counter variable
    index_name = f"{base_index_name}_{doc_count // max_docs_per_index}"

    # Check if the index exists, and create it if it doesn't
    if not client.indices.exists(index=index_name):
        client.indices.create(index=index_name)

    # Index the document in the current index
    client.index(index=index_name, body=doc)

    # Increment the counter variable
    doc_count += 1

/var/folders/w3/d1c_m0dx0mv20q7033lt172h0000gn/T/ipykernel_9298/2358095211.py:12: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.14/security-minimal-setup.html to enable security.
  if not client.indices.exists(index=index_name):
/var/folders/w3/d1c_m0dx0mv20q7033lt172h0000gn/T/ipykernel_9298/2358095211.py:13: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.14/security-minimal-setup.html to enable security.
  client.indices.create(index=index_name)
/var/folders/w3/d1c_m0dx0mv20q7033lt172h0000gn/T/ipykernel_9298/2358095211.py:16: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See ht

KeyboardInterrupt: 

In [655]:

settings = {
    "settings": {
        "index": {
            "mapping": {
                "total_fields": {
                    "limit": 2000,
                "properties": {
                    "place": {
                        "type": "Object"
                    },
                    "entities": {
                        "type": "Object"
                    },
                    "extended_entities": {
                        "type": "Object"
                    },
                    "quoted_status": {
                        "type": "Object"
                    },
                    "retweeted_status": {
                        "type": "Object"
                    },
                    "text": {
                        "type": "Object"
                    },
                    "source": {
                        "type": "Object"
                    },
                    "lang": {
                        "type": "Object"
                    },
                    "id_str": {
                        "type": "Integer"
                    },
                    "user_id": {
                        "type": "Integer"
                    },
                    "quote_count": {
                        "type": "Integer"
                    },
                    "reply_count": {
                        "type": "Integer"
                    },
                    "retweet_count": {
                        "type": "Integer"
                    },
                    "favorite_count": {
                        "type": "Integer"
                    },
                    "possibly_sensitive": {
                        "type": "Float"
                    },
                    "retweeted": {
                        "type": "Boolean"
                    },
                    "truncated": {
                        "type": "Boolean"
                    },
                    "favorited": {
                        "type": "Boolean"
                    },
                    # "created_at": {
                    #     "type": "Date",
                    #     "format": "epoch_millis"
                    # },
                    "withheld_in_countries": {
                        "type": "Object"
                    }
                }
                }
            }
        }
    }
}

client.indices.create(index='tweet_index-000001', body=settings, request_timeout=30)
client.indices.put_alias(index='tweet_index-000001', name='tweet_data')

# Index streaming data with rollover
i = 0
for data in docs:
    

    # Index document with rollover
    client.index(index='tweet_data', body=data)
    i += 1
    if i % 1000 == 0:
        client.indices.rollover(alias='tweet_data')

/var/folders/w3/d1c_m0dx0mv20q7033lt172h0000gn/T/ipykernel_9298/4131612587.py:76: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  client.indices.create(index='tweet_index-000001', body=settings, request_timeout=30)
/var/folders/w3/d1c_m0dx0mv20q7033lt172h0000gn/T/ipykernel_9298/4131612587.py:76: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  client.indices.create(index='tweet_index-000001', body=settings, request_timeout=30)


BadRequestError: BadRequestError(400, 'illegal_argument_exception', 'unknown setting [index.mapping.total_fields.properties.entities.type] please check that any required plugins are installed, or check the breaking changes documentation for removed settings')

In [656]:
index_names = ['index1', 'index2', 'index3','index4', 'index5', 'index6']

# Create the indices
for index_name in index_names:
    client.indices.create(index=index_name)

# Define the documents to be indexed


# Index the documents in the corresponding indices
for i, doc in enumerate(docs):
    index_name = index_names[i % len(index_names)]
    client.index(index=index_name, body=doc)

/var/folders/w3/d1c_m0dx0mv20q7033lt172h0000gn/T/ipykernel_9298/2048099060.py:5: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.14/security-minimal-setup.html to enable security.
  client.indices.create(index=index_name)
/var/folders/w3/d1c_m0dx0mv20q7033lt172h0000gn/T/ipykernel_9298/2048099060.py:13: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  client.index(index=index_name, body=doc)
/var/folders/w3/d1c_m0dx0mv20q7033lt172h0000gn/T/ipykernel_9298/2048099060.py:13: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsea

BadRequestError: BadRequestError(400, 'mapper_parsing_exception', 'failed to parse')

In [666]:
doc

{'id_str': 1254022922458587136,
 'text': '@jtucksh0p A name I saw a lot before Corona. Tempted to go in now',
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'user_id': 1217083010308362242,
 'truncated': False,
 'lang': 'en',
 'quote_count': 0,
 'reply_count': 0,
 'retweet_count': 0,
 'favorite_count': 0,
 'favorited': False,
 'retweeted': False,
 'possibly_sensitive': None,
 'withheld_in_countries': None}

In [ ]:
client.indices.delete(index='_all')

/var/folders/w3/d1c_m0dx0mv20q7033lt172h0000gn/T/ipykernel_9298/3743574261.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.14/security-minimal-setup.html to enable security.
  client.indices.delete(index='_all')
/var/folders/w3/d1c_m0dx0mv20q7033lt172h0000gn/T/ipykernel_9298/3743574261.py:1: ElasticsearchWarning: setting [action.destructive_requires_name] will default to true in 8.0, set explicitly to false to preserve current behavior
  client.indices.delete(index='_all')


ObjectApiResponse({'acknowledged': True})

In [627]:
doc

{'id_str': 1249403797064921088,
 'created_at': 1586716052000,
 'text': 'RT @TheLostPiraTE_: Kapde phen le bhai nhi to G se corona ghus jayega hawa bhi nhi lagegi',
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'user_id': 1220391594295205889,
 'truncated': False,
 'lang': 'hi',
 'quote_count': 0,
 'reply_count': 0,
 'retweet_count': 0,
 'favorite_count': 0,
 'favorited': False,
 'retweeted': False,
 'possibly_sensitive': None,
 'withheld_in_countries': None,
 'place': None,
 'entities': {'hashtags': [],
  'urls': [],
  'user_mentions': [{'screen_name': 'TheLostPiraTE_',
    'name': 'Mr. Psychopath',
    'id': 43710217,
    'id_str': '43710217',
    'indices': [3, 18]}],
  'symbols': []},
 'extended_entities': None,
 'quoted_status': {'created_at': 'Sun Apr 12 17:24:23 +0000 2020',
  'id': 1249387903018389504,
  'id_str': '1249387903018389504',
  'text': '@Modern_Manthra Maafi maango #COVID__19 se',
  'display_text_range': [16, 42],
 

In [651]:
type(docs[0]['favorited'])

bool

In [709]:
[i for i in twitterdf.loc[twitterdf['entities'].notna(),'entities'].tolist() if i['hashtags'] != []]

[{'hashtags': [{'text': 'Bo', 'indices': [0, 3]}],
  'urls': [],
  'user_mentions': [],
  'symbols': []},
 {'hashtags': [{'text': 'pazar', 'indices': [105, 111]},
   {'text': 'COVID2019', 'indices': [112, 122]},
   {'text': 'Corona', 'indices': [123, 130]}],
  'urls': [],
  'user_mentions': [{'screen_name': 'ercumentovalii',
    'name': 'Prof.Dr. Ercüment Ovalı',
    'id': 1212623185373327361,
    'id_str': '1212623185373327361',
    'indices': [3, 18]}],
  'symbols': []},
 {'hashtags': [{'text': 'COVID19', 'indices': [32, 40]},
   {'text': 'HerdImmunity', 'indices': [41, 54]},
   {'text': 'NHS', 'indices': [56, 60]}],
  'urls': [],
  'user_mentions': [{'screen_name': 'MattHancock',
    'name': 'Matt Hancock',
    'id': 19825835,
    'id_str': '19825835',
    'indices': [0, 12]},
   {'screen_name': '10DowningStreet',
    'name': 'UK Prime Minister',
    'id': 14224719,
    'id_str': '14224719',
    'indices': [13, 29]}],
  'symbols': []},
 {'hashtags': [{'text': 'DateMyFamily', 'indice

In [ ]:
# Compute the index name based on the counter variable
doc = docs[1000]
doc_count = 51

index_name = f"{base_index_name}_{doc_count  // max_docs_per_index}"

# Check if the index exists, and create it if it doesn't
if not client.indices.exists(index=index_name):
    client.indices.create(index=index_name)

# Index the document in the current index
client.index(index=index_name, body=doc)

# Increment the counter variable
doc_count += 1

In [ ]:
index_name

In [ ]:
client.indices.create(index='twit4')
client.index(index='twit4', body=docs[0])

In [ ]:
search_query = {}
count_results = client.count(index='twitter_data_0', body=search_query)

# extract the document count from the count results
document_count = count_results['count']
# make the search request and retrieve the results
# search_results = client.search(index='twitter_data_1', body=search_query)

# extract the documents from the search results
# documents = [hit['_source'] for hit in search_results['hits']['hits']]

In [ ]:
# documents
document_count


In [ ]:
docs[:100]

In [710]:
client.indices.delete(index='_all')

/var/folders/w3/d1c_m0dx0mv20q7033lt172h0000gn/T/ipykernel_9298/3743574261.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.14/security-minimal-setup.html to enable security.
  client.indices.delete(index='_all')
/var/folders/w3/d1c_m0dx0mv20q7033lt172h0000gn/T/ipykernel_9298/3743574261.py:1: ElasticsearchWarning: setting [action.destructive_requires_name] will default to true in 8.0, set explicitly to false to preserve current behavior
  client.indices.delete(index='_all')


ObjectApiResponse({'acknowledged': True})

In [ ]:
aliases = client.indices.get_alias()

# count the number of indices
index_count = len(aliases)
index_count

In [ ]:
docs[47]